In [8]:
from o9_common_utils.O9DataLake import O9DataLake, ResourceType, DataSource, PluginSetting
sales = O9DataLake.register("SalesActual",DataSource.LIVEFRAME,ResourceType.LIVEFRAME, plugin_setting = PluginSetting.Inputs)
output = O9DataLake.register("output",data_source = DataSource.LS,entity_type = ResourceType.IBPL, plugin_setting = PluginSetting.Outputs)

In [9]:
O9DataLake.inputs

{}

In [7]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import numpy as np
import logging

from o9_flow.O9Flow import O9Flow

logger = logging.getLogger('o9_logger')

logger.info("sample plugin template")
logger.info('o9flow initialising')

df = sales

data = df.rename(columns={'Time.[Week]': 'Week', 'Item.[Planning Item]': 'Item', 'Location.[Location]':'Location'})

data['Week'] = pd.to_datetime(data['Week'])
data.set_index('Week', inplace=True)

items = data['Item'].unique()
locations = data['Location'].unique()

predictions_df = pd.DataFrame(columns=['Item', 'Location', 'Week', 'Predicted Sales'])



logger.info(data.head())

for item in items:
    for location in locations:
        # Filter data for the specific item and location
        subset_data = data[(data['Item'] == item) & (data['Location'] == location)]
        if subset_data.empty != True:
            item_location_sales = subset_data['AIPActual']

        # Calculate Simple Moving Average
            window_size = 3  # Adjust the window size as needed
            sma = item_location_sales.rolling(window=window_size).mean()

            # Make predictions for the next 10 weeks (assuming a constant average)
            average_prediction = sma.iloc[-1]   # Use the last calculated average

            # Create a DataFrame with the predictions
            predictions_subset = pd.DataFrame({
                'Item': [item] * 10,
                'Location': [location] * 10,
                'Week': pd.date_range(start=item_location_sales.index[-1] + pd.DateOffset(1), periods=10, freq='W'),
                'Predicted Sales': [average_prediction] * 10
            })

            # Append the predictions to the overall DataFrame
            predictions_df = predictions_df.append(predictions_subset, ignore_index=True)

            
            predictions_df['Week'] = pd.to_datetime(predictions_df['Week']).dt.strftime('%d-%b-%y')

o9flow.log_params(params= {'param_A':8, 'param_B':1.5, 'param_C':'Testing'})
o9flow.log_metrics(metrics={'plugin_feature_A': 4.99, 'plugin_feature_B': 3.00}, step=11)
            
output = predictions_df.rename(columns={'Week':'Time.[Week]', 'Item':'Item.[Planning Item]', 'Location':'Location.[Location]'})
output['Version.[Version Name]'] = 'CurrentWorkingView'
List = ['Version.[Version Name]','Item.[Planning Item]','Location.[Location]','Time.[Week]','Predicted Sales']
output = output[List]

logger.info(output.head())


2024-01-16 12:59:37,408 - o9_logger - INFO  - sample plugin template
2024-01-16 12:59:37,409 - o9_logger - INFO  - o9flow initialising


AttributeError: 'str' object has no attribute 'rename'

In [6]:
O9DataLake.put('output', output)

In [ ]:
import logging
logger = logging.getLogger('o9_logger')
logger.info('o9flow initialising')
from o9_flow.O9Flow import O9Flow
import numpy as np
import matplotlib.pyplot as plt
figure_value = plt.figure()
figure_value.add_axes([10,10,10,10]) # plotting graph can be stored as an figure in o9Flow
image_value = np.ndarray(shape=(2,2), dtype=float, order='F') 
o9flow = O9Flow()
o9flow.start_run(run_name = 'T010')
o9flow.log_text(text = 'Testing Plugin', artifact_file='Plugin_testing_log_text.txt')
o9flow.log_dict(dictionary={'param1':'Integer', 'param2':'Float'}, artifact_file='Plugin_testing_log_dict.json')
o9flow.log_figure(figure=figure_value, artifact_file='Plugin_testing_log_figure.png')
o9flow.log_image(image=image_value, artifact_file='Plugin_testing_log_img.png')
o9flow.log_params(params= {'param_A':8, 'param_B':1.5, 'param_C':'Testing'})
o9flow.log_metric(key='plugin_feature', value= 6.0, step=2) # metric can be stored as a float value
o9flow.log_metrics(metrics={'plugin_feature_A': 4.99, 'plugin_feature_B': 3.00}, step=11)
o9flow.log_metrics(metrics={'plugin_feature_A': 4.8, 'plugin_feature_B': 3.8}, step=3)
o9flow.log_metrics(metrics={'plugin_feature_N': 0.8, 'plugin_feature_B': .8}, step=2)



o9flow.log_metrics(metrics={'plugin_feature_A': 4.8, 'plugin_feature_B': 3.8}, step=3)
o9flow.log_metrics(metrics={'plugin_feature_N': 0.8, 'plugin_feature_B': .8}, step=2)
o9flow.log_text(text = 'Testing Plugin', artifact_file='Plugin_testing_log.txt')
o9flow.log_dict(dictionary={'param1':'gufyuiugbubbuhi', 'param2':'t7ggtt7uuygy7y'}, artifact_file='Plugin_testing_log_dict.json')
o9flow.end_run()